# Measuring the Z boson mass


Let's look at a sample of $Z$ boson candidates recorded by CMS in 2011 and published at CERN opendata portal. It comes from DoubleMuon dataset with the following selection applied:

- Both muons are "global" muons
- invariant mass sits in range: 60 GeV < $ M_{\mu\mu}$ < 120 GeV
- |$\eta$| < 2.1 for both muons
- $p_{t}$ > 20 GeV

The following columns presented in the CSV file:

- `Run`, Event are the run and event numbers, respectively
- `pt` is the transverse momentum $p_{t}$ of the muon
- `eta` is the pseudorapidity of the muon: $\eta$
- `phi` is the $\phi$ angle of the muon direction
- `Q` is the charge of the muon
- `dxy` is the impact parameter in the transverse plane: $d_{xy}$ - how distant is the track from the collision point
- `iso` is the track isolation: $I_{track}$ - how many other tracks are there around a given track

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skopt import gp_minimize
%matplotlib inline

## Read dataset

In [ ]:
df = pd.read_csv('./Zmumu.csv')
df.shape

Let's calculate the invariant mass $M$ of the two muons using the formula

$M = \sqrt{2p_{t}^{1}p_{t}^{2}(\cosh(\eta_{1}-\eta_{2}) - \cos(\phi_{1}-\phi_{2}))}$

In [ ]:
df['M'] = np.sqrt(2 * df['pt1'] * df['pt2'] * (np.cosh(df['eta1'] - df['eta2']) - np.cos(df['phi1'] - df['phi2'])))
df.head(2)

# Model

The distribution of the Z boson mass has the form of a normal distribution, in addition there is a noise. The distribution of the noise has an exponential form. Thus, the resulting model, it is a result of a superposition of two distributions - normal and polinomial

Let's plot the distribution of Z boson mass

In [ ]:
def plot_mass(mass, bins_count=100):
    """
    Parameters
    ----------
    """
    y, x = np.histogram(mass, bins=bins_count, density=False)
    err = np.sqrt(y)

    fig = plt.figure(figsize=(15,7))
    plt.title('Z mass', fontsize=20)
    plt.xlabel("$m_{\mu\mu}$ [GeV]", fontsize=20)
    plt.ylabel("Number of events", fontsize=20)
    plt.errorbar(x[:-1], y, yerr=err, fmt='o', color='red', ecolor='grey', capthick=0.5, zorder=1, label="data")
    return y, x

In [ ]:
plot_mass(df.M);

## Exercise 1. clean up dataset a bit
- demand that charge of muons should be opposite
- $I_{track}$ < 3 and $d_{xy}$ < 0.2 cm

In [ ]:
df.describe()

In [ ]:
# NOTE: Opposite sign of the charges
# NOTE: It looks like the measured charges are not integers
df = df.loc[((df.loc[:, 'Q1'] < 0) & (df.loc[:, 'Q2'] > 0)) |
            ((df.loc[:, 'Q1'] > 0) & (df.loc[:, 'Q2'] < 0))]
# NOTE: dxy is given in cm
df = df.loc[(df.loc[:, 'dxy1'].abs() < 0.2) & (df.loc[:, 'dxy2'].abs() < 0.2)]
# NOTE: Both tracks should have I_track < 3
df = df.loc[(df.loc[:, 'iso1'] < 3) & (df.loc[:, 'iso2'] < 3)]

In [ ]:
df.shape

In [ ]:
plot_mass(df.M);

### Let's define a parametrised model
it should represent mixture of

1. gaussian signal and 
2. background that for the simplicity we consider to be flat over mass

So it gives the following set of parameters:

- m0 - center of the Gaussian
- sigma - standard deviation of the Gaussian
- ampl - height of the peak
- bck - height of the background 

finding those parameters is called _fitting_ model into the data. It will be the goal for the rest of the exercise. For simplicity sake we'll stick with old good binned fit.

In [ ]:
def model_predict(params, X):
    """
    
    Parameters
    ----------
    
    Returns
    -------
    """
    
    m0, sigma, ampl, bck = params
    
    return bck + ampl / (sigma * np.sqrt(2 * np.pi)) * np.exp((-1) * (X - m0)**2 / (2 * sigma**2))

In [ ]:
def model_loss(params, X, y):
    """
    
    Parameters
    ----------
    
    Returns
    -------
    """

    residuals = y - model_predict(params, X)
    
    return np.sum(residuals**2) / len(residuals)

In [ ]:
def plot_mass_with_model(params, mass, bins_count=100):
    """
    
    Parameters
    ----------
    """
    
    y, X = plot_mass(mass, bins_count=bins_count)
    X = (X[1:] + X[:-1]) / 2
    error = model_loss(params, X, y)
    plt.plot(X, model_predict(params, X), color='blue', linewidth=3.0, zorder=2, label="fit, loss=%.2f" % error)
    plt.legend(fontsize='x-large')

## Here you can fit model parameters by hand

In [ ]:
plot_mass_with_model((75, 5, 2300, 20), df.M)

## ... but you can do it automatically of course

Setting up a scikit optimizer

In [ ]:
from tqdm import tqdm
from skopt import Optimizer

search_space = [(30.0, 180.0), # m0 range
    (0.5, 10.0), # sigma range
    (2000, 8000), # amplitude range
    (0, 50) # bck range
    ]
y, X = np.histogram(df.M, bins=120, density=False)
X = (X[1:] + X[:-1]) / 2
opt = Optimizer(search_space, base_estimator="GP", acq_func="EI", acq_optimizer="lbfgs")

Running it for a while. You can re-run this cell several times

In [ ]:
from skopt.utils import create_result
for i in tqdm(range(50)):
    next_x = opt.ask()
    f_val = model_loss(next_x, X, y)
    opt.tell(next_x, f_val)
    
res = create_result(Xi=opt.Xi, yi=opt.yi, space=opt.space,
                         rng=opt.rng, models=opt.models)

## A bit of search history

In [ ]:
import skopt.plots
skopt.plots.plot_convergence(res)
print (list(zip(["m0", "sigma", "ampl", "bck"], res.x)))

In [ ]:
# even more details on the search space
# skopt.plots.plot_objective(res, dimensions=['m0', 'sigma', 'ampl', 'bck'])


Let's see how well the prediction fits the data

In [ ]:
plot_mass_with_model(res.x, df.M, bins_count=120)

In [ ]:
# token expires every 30 min
COURSERA_TOKEN = "aAja9pbfyRMtibne"### YOUR TOKEN HERE
COURSERA_EMAIL = "andrey.u@gmail.com"### YOUR EMAIL HERE

## Grader part, do not change, please

In [ ]:
import grading
grader = grading.Grader(assignment_key="SeMsm2JfEeiyvBKA0wfIgA", 
                      all_parts=["VI3xu", "VuE8x", "KzmMV", "TwZBF"])


In [ ]:
ans_part1 = round(res.x[0])
grader.set_answer("VI3xu", ans_part1)

ans_part2 = round(res.x[1], 2)
grader.set_answer("VuE8x", ans_part2)

ans_part3 = round(res.x[3])
grader.set_answer("KzmMV", ans_part3)

ans_part4 = round(res.x[2])
grader.set_answer("TwZBF", ans_part4)

grader.submit(COURSERA_EMAIL, COURSERA_TOKEN)